In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pickle
import pretty_midi
import librosa
import librosa.display
import gc
from sklearn.preprocessing import StandardScaler
import warnings
from collections import Counter
from torch.utils.data import Dataset
import torch
from pympler import asizeof
import torch.optim as optim 



import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset


from sklearn.preprocessing import StandardScaler

from Preprocessing import *
#from ExtractGenre import *
from CNN_ExtractGenre import *
from PolyphonicPreprocessing import *
import Util as Util

import DatasetLoader as DL
import Model as M



In [ ]:
GenreMapping = {'metal': 0, 'disco': 1, 'classical': 2, 'hiphop': 3, 'jazz': 4,
          'country': 5, 'pop': 6, 'blues': 7, 'reggae': 8, 'rock': 9}

In [ ]:
Path = os.path.realpath('clean_midi/Eminem/Stan.mid')
mid = mido.MidiFile(Path)
#print(mid)



In [ ]:
Data = DL.MonophonicDataset(Instruments=['Bass'], EightBars=True)
Bars = DataLoader(Data, batch_size=10, shuffle=True, num_workers=0)

# 1 Instrument 

In [3]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print('GPU available')
else:
    device = torch.device("cpu")
    print('GPU not available')


def LoadModel(Cond1D_Size, instrumentSize, Which):

    Path = os.path.realpath('ModelParameters')

    files = [
        ['generator_parameters.torch', 'gen_opt_state.torch'],
        ['generator_parameters.torch', 'gen_opt_state.torch']
    ]

    generator = M.Generator(input_size=256, cond_1d_size=Cond1D_Size, instrument_size=instrumentSize, n_hlayers=256)
    generator.apply(M.weights_init)
    generator.to(device)
    # Load the state dict previously saved
    generator_state_dict = torch.load(os.path.join(Path, files[Which][0]), map_location=torch.device('cpu'))
    # Update the network parameters
    generator.load_state_dict(generator_state_dict)

    gen_opt = optim.Adam(generator.parameters(), lr=2e-4)
    # Load the state dict previously saved
    gen_opt_state_dict = torch.load(os.path.join(Path, files[Which][1]), map_location=torch.device('cpu'))
    # Update the network parameters
    gen_opt.load_state_dict(gen_opt_state_dict)

    return generator

                                                    #MOnophonic or polyphonic
generator = LoadModel(Cond1D_Size=5, instrumentSize=4, Which=1)

GPU not available


In [4]:
PolyDataset = PolyphonicPreProcessing(nDir = 50)

Preprocessing: 100%|██████████| 150/150 [00:32<00:00,  4.59it/s]


In [5]:
TEMPO_MIN, TEMPO_MAX = 60, 200  # Typical tempo range
PROGRAM_MIN, PROGRAM_MAX = 1, 128  # MIDI program range

def NormCond(tempo, programs):
    # Normalize tempo to [0, 1]
    tempo_norm = (tempo - TEMPO_MIN) / (TEMPO_MAX - TEMPO_MIN)
    
    # Normalize programs to [0, 1]
    programs_norm = [(p - PROGRAM_MIN) / (PROGRAM_MAX - PROGRAM_MIN) for p in programs]
    
    return [tempo_norm] + programs_norm

In [16]:
generator.eval()

Genre = 'rock'
bar = np.random.randint(0, 100)

prev_bar = PolyDataset[Genre][bar]['Bars'][0].to_dense().float().to(device)
InstrumentCode = PolyDataset[Genre][bar]['Program'][0]
Tempo = PolyDataset[Genre][bar]['Tempo'][0]

cond_1d = torch.tensor([NormCond(Tempo, InstrumentCode)], dtype= torch.float32)
Bars = []
Bars.append(prev_bar)
#If polyphonic only 1 unsqueeze
prev_bar = prev_bar.unsqueeze(0)#.unsqueeze(0) 


for i in range(8):
   noise = torch.rand([1, 256], device=device)

   #print(np.mean(np.array(noise)))

   with torch.no_grad():
      generated_bar = generator(noise, prev_bar, cond_1d, 1)

   binary_bar = (generated_bar > 0.4).float()  # still a tensor
   Bars.append(binary_bar.squeeze(0).cpu().numpy())  # only now for MIDI
   print(generated_bar.mean().item(), generated_bar.std().item())
   prev_bar = binary_bar.detach()

   if i % 2 == 0:
      prev_bar = binary_bar.detach()
   else:
      prev_bar = PolyDataset[Genre][bar + i]['Bars'][0].to_dense().float().to(device).unsqueeze(0)




#ConcBars = np.concatenate(Bars, axis = 1)
PolyConcBars = np.concatenate(Bars, axis = 2)

PolyBarsToMIDI(PolyConcBars, title='Polytest', Instrument=InstrumentCode)

print(InstrumentCode)


0.005938210524618626 0.045797038823366165
0.0052363937720656395 0.04101745784282684
0.0057996888644993305 0.04395905137062073
0.006194867193698883 0.04693875461816788
0.009649202227592468 0.05970766767859459
0.012444048188626766 0.08934357017278671
0.010406783781945705 0.0649520605802536
0.01269016694277525 0.09213338792324066
[1, 33, 49, 25]


In [ ]:
polconc = []
for i in range(50, 54):
   for j in range(2):
      polconc.append(PolyDataset['rock'][i]['Bars'][j].to_dense())

conc = np.concatenate(polconc, axis = 2)
PolyBarsToMIDI(conc, title = 'True', Instrument = [33, 41, 49, 73])

In [ ]:


torch.randn(np.shape(torch.ones_like(prev_bar)))*0.2